In [ ]:
# A helper function that takes a Hugging Face Dataset, the name of the corresponding sentence field on that Dataset,
# the name of the label field on Dataset and a Tokenizer to output a tokenized TensorDataset:
# Source: DL with Python, sbs - Godoy, Vol2. pág.: 1119.

In [ ]:
def tokenize_dataset(hf_dataset, sentence_field, label_field, tokenizer, **kwargs):
    
    sentences  = hf_dataset[sentence_field]
    tokens_ids = tokenizer(sentences, return_tensors='pt', **kwargs)['input_ids']
    
    labels     = torch.as_tensor(hf_dataset[label_field])
    
    dataset    = TensorDataset(token_ids, labels)
    
    return dataset

In [ ]:
#modelo          = 'distilbert-base-uncased'

In [ ]:
auto_tokenizer  = AutoTokenizer.from_pretrained(modelo)

In [ ]:
tkzr_kwargs     = dict(truncation=True, padding=True, max_length=30, add_special_tokens=True)

In [ ]:
train_tensor_ds = tokenize_dataset(train_dataset, 'sentence', 'label', auto_tokenizer, **tkzr_kwargs)
test_tensor_ds  = tokenize_dataset(test_dataset, 'sentence', 'label', auto_tokenizer, **tkzr_kwargs)

generator       = torch.Generator()

train_loader    = DataLoader(train_tensor_ds, batch_size=4, suffle=True, generator=generator)
test_loader     = DataLoader(test_tensor_ds, batch_size=8)

In [ ]:
# Another variation is:
# Source: page 1127

auto_tokenizer     = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize(row):
    return auto_tokenizer(row['sentence_field'],truncation=True, padding=max_length, max_length=30)

tokenized_train_ds = train_dataset.map(tokenize, batched=True)
tokenized_test_ds  = test_dataset.map(tokenize, batched=True)

In [ ]:
def compute_metrics(eval_pred):
    predictions    = np.argmax(eval_pred.predictions, axis=1)
    labels         = eval_pred.label_ids
    return {"Accuracy: ": (predictions == labels).mean()}

In [ ]:
from transformers import Trainer, TrainingArguments
training_args      = TrainingArguments(
    output_dir     = 'output',
    num_train-epochs = 1,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 8,
    evaluation_strategy = 'steps',
    eval_steps     = 300,
    loggin_steps   = 300,
    gradient_accumulation_steps = 8
)

trainer            = Trainer(model=bert_cls, 
                             args=training_args
                             train_datset=tokenized_train_ds,
                             eval_dataset=tokenized_test_ds,
                             compute_metrics=compute_metrics)

trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model('MyBert')
os.listdir('MyBert')

In [ ]:
# See predictions on 1135 page!

In [1]:
from transformers.pipelines import SUPPORTED_TASKS
SUPPORTED_TASKS

In [ ]:
# Loading a dataset from a CSV file:
# Source: page 1145
dataset            = load_dataset(path='csv',
                      data_files=['path/to/csv.csv'],
                      quotechar='\\',
                      split=Split.TRAIN)

shuffled_dataset   = dataset.shuffle(seed=42)

split_dataset      = shuffled_dataset.train_test_split(test_size=0.2, seed=42)

train_dataset      = split_dataset['train']
test_dataset       = split_dataset['test']
